In [1]:
!pip install pdfplumber pandas openpyxl


     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.8 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 516.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
     ---------------------------------------- 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/60.0 kB ? eta -:--:--
   ---------------------------------------- 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.6 MB 15.5 MB/s eta 0:00:01
   --------------- ------------------------ 2.1/5.6 MB 22.8 MB/s eta 0:00:01
   ----------------------------------- ---- 5.0/5.6 MB 40.2 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 32.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   -------------------

In [2]:
import pdfplumber
import pandas as pd

In [3]:
with pdfplumber.open("DBI_FirstQ.pdf") as pdf:   
    page = pdf.pages[12]
    # Extract all tables from that page
    tables = page.extract_tables()

    # Loop through all tables and print to find Canada Retail data
    for i, table in enumerate(tables):
        print(f"\n--- Table {i} ---")
        for row in table:
            print(row)


--- Table 0 ---
['Net sales:', '', None, '', '', None]
['U.S. Retail segment:', '', None, '', '', None]
['Non-athletic footwear:', '', None, '', '', None]
["Women's", '$', '275,526', '', '$', '306,354']
["Men's", '70,935', None, '', '75,798', None]
["Kids'", '21,471', None, '', '21,686', None]
['Athletic footwear', '175,165', None, '', '184,525', None]
['Accessories and other', '30,143', None, '', '33,004', None]
['', '573,240', None, '', '621,367', None]
['Canada Retail segment:', '', None, '', '', None]
['Non-athletic footwear:', '', None, '', '', None]
["Women's", '19,207', None, '', '20,823', None]
["Men's", '7,509', None, '', '7,418', None]
["Kids'", '2,963', None, '', '2,821', None]
['Athletic footwear', '22,064', None, '', '22,476', None]
['Accessories and other', '2,162', None, '', '1,974', None]
['', '53,905', None, '', '55,512', None]
['Brand Portfolio segment:', '', None, '', '', None]
['Wholesale', '84,498', None, '', '88,670', None]
['Direct-to consumer', '10,355', None, 

In [5]:
import pandas as pd

# Extract Canada Retail rows
table = tables[0]

# Find start and end of the Canada Retail segment
start_index = next(i for i, row in enumerate(table) if row[0] and "Canada Retail segment" in row[0])
end_index = next(i for i, row in enumerate(table) if row[0] and "Brand Portfolio segment" in row[0])

# Slice the rows in between
canada_rows = table[start_index + 2:end_index]  # Skip label lines

# Trim rows to only 3 values: Category, Q1_2025, Q1_2024
cleaned_rows = []
for row in canada_rows:
    row_clean = [cell for cell in row if cell not in [None, '', '$']]
    if len(row_clean) >= 3:
        cleaned_rows.append(row_clean[:3])

# Create DataFrame
df_canada = pd.DataFrame(cleaned_rows, columns=["Category", "Q1_2025", "Q1_2024"])

# Save to Excel
df_canada.to_excel("Canada_Retail_Q1_Sales.xlsx", index=False)


In [6]:
import pdfplumber
import pandas as pd

with pdfplumber.open("DBI_FirstQ.pdf") as pdf:
    page = pdf.pages[12]  # Page 13 in PDF file
    table = page.extract_tables()[0]

# Extract sections from the large table
def extract_segment(table, start_label, end_label=None):
    start = next(i for i, row in enumerate(table) if row[0] and start_label in row[0])
    if end_label:
        end = next(i for i, row in enumerate(table) if row[0] and end_label in row[0])
    else:
        end = len(table)
    rows = table[start + 2:end]
    cleaned = []
    for row in rows:
        values = [cell for cell in row if cell not in [None, '', '$']]
        if len(values) >= 3:
            cleaned.append(values[:3])
    return cleaned

# Extract each section
us_rows = extract_segment(table, "U.S. Retail segment:", "Canada Retail segment:")
canada_rows = extract_segment(table, "Canada Retail segment:", "Brand Portfolio segment:")
brand_rows = extract_segment(table, "Brand Portfolio segment:", "Total segment net sales")
summary_rows = extract_segment(table, "Total segment net sales", None)

# Convert to DataFrames
df_us = pd.DataFrame(us_rows, columns=["Category", "Q1_2025", "Q1_2024"])
df_canada = pd.DataFrame(canada_rows, columns=["Category", "Q1_2025", "Q1_2024"])
df_brand = pd.DataFrame(brand_rows, columns=["Category", "Q1_2025", "Q1_2024"])
df_summary = pd.DataFrame(summary_rows, columns=["Category", "Q1_2025", "Q1_2024"])

# Save everything to one Excel with multiple sheets
with pd.ExcelWriter("DesignerBrands_Q1_Sales_Full.xlsx") as writer:
    df_us.to_excel(writer, sheet_name="US Retail", index=False)
    df_canada.to_excel(writer, sheet_name="Canada Retail", index=False)
    df_brand.to_excel(writer, sheet_name="Brand Portfolio", index=False)
    df_summary.to_excel(writer, sheet_name="Totals", index=False)
